In [17]:
import pandas as pd

# Read the dataset
df = pd.read_csv("F:/Works/Data Visualisation/Datasets/Motor_Vehicle_Collisions_-_Crashes.csv", low_memory=False)

# Remove rows with blank values in the "LOCATION", "ZIP CODE", "BOROUGH", "CONTRIBUTING FACTOR VEHICLE 1" and "VEHICLE TYPE CODE 1" columns
df = df.dropna(subset=['LOCATION', 'ZIP CODE', 'BOROUGH', 'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1'])

# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

# Drop irrelevant contributing factor and vehicle type code columns
columns_to_drop = ['CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 
                   'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 
                   'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
df = df.drop(columns=columns_to_drop)

# Convert "CRASH DATE" and "CRASH TIME" columns to datetime format
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'], errors='coerce')
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M', errors='coerce').dt.time

# Categorize time of day based on "CRASH TIME"
def categorize_time(time):
    if pd.Timestamp('00:00:00').time() <= time <= pd.Timestamp('05:59:59').time():
        return '12:00 am to 5:59 am'
    elif pd.Timestamp('06:00:00').time() <= time <= pd.Timestamp('10:59:59').time():
        return '6 am to 10:59 am'
    elif pd.Timestamp('11:00:00').time() <= time <= pd.Timestamp('17:59:59').time():
        return '11 am to 5:59 pm'
    else:
        return '6 pm to 11:59 pm'

df['Time of Day'] = df['CRASH TIME'].apply(categorize_time)

# Map day of week to "Weekday" or "Weekend"
def map_day_to_type_of_day(day):
    if day < 5:  # Monday to Friday (0-4)
        return 'Weekday'
    else:       # Saturday and Sunday (5-6)
        return 'Weekend'

df['Type of Day'] = df['CRASH DATE'].dt.dayofweek.apply(map_day_to_type_of_day)

# Categorize time as "Daytime" or "Nighttime"
def categorize_day_or_night(crash_time):
    if crash_time.hour >= 6 and crash_time.hour < 18:
        return 'Daytime'
    else:
        return 'Nighttime'

df['Day/Night'] = df['CRASH DATE'].apply(categorize_day_or_night)

# Save the pre-processed dataset
df.to_csv("F:/Works/Data Visualisation/Datasets/motor_collision_cleaned.csv", index=False)